## Interactively Export Model to ONNX

In [9]:
import os
import time

from typing import Iterable
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm

from torchvision import datasets, transforms

from torch.optim import lr_scheduler
from torch import tensor
from tqdm.notebook import tqdm

from utils import *
from learner import *
from data import *
from models import *
from criterion import *
from config import *

from dataloaders.nyu import NYUDataset

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext tensorboard
%reload_ext autoreload
%autoreload 2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [10]:
plt.rcParams['figure.figsize'] = (20, 20)

In [11]:
config = Config()
config.write()

In [12]:
from torch_lr_finder import LRFinder

In [13]:
torch.manual_seed(config.seed)
np.random.seed(config.seed)
random.seed(config.seed)

In [14]:
MODEL_PATH = 'all_data_91.pth'

In [15]:
model = MobileNetV2SkipAdd(pretrained=True, interpolation='bilinear')

In [16]:
model.load_state_dict(torch.load(MODEL_PATH, map_location=torch.device('cpu'))['state_dict'])

<All keys matched successfully>

In [17]:
model.eval()

MobileNetV2SkipAdd(
  (conv0): ConvBNReLU(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
  )
  (conv1): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (conv2): InvertedResidual(
    (conv): Sequential(
      (0): ConvBNReLU(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(

In [18]:
class Normalizer(nn.Module):
    def __init__(self):
        super(Normalizer, self).__init__()
    def forward(self, x):
        min_rows = x.min(0)
        min = min_rows.min(1)
        max_rows = x.max(0)
        max = max_rows.max(1)
        return 255. * (x - min)/(max - min)
    
def export_model(model, outpath='depth.onnx', inputnames=['images'], outputnames=['depths'], normalize=True):
    dummy_input = torch.randn(1, 3, 224, 224)
    if normalize:
        model = nn.Sequential(model, Normalizer())
    torch.onnx.export(model, dummy_input, outpath, verbose=True, input_names=inputnames, output_names=outputnames)
    

In [19]:
export_model(model, 'depth_test.onnx', normalize=False)

/Users/dhritiman.sagar/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/Users/dhritiman.sagar/Snapchat/Dev/DepthNet/models.py:912: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  print(x.shape)
/Users/dhritiman.sagar/Snapchat/Dev/DepthNet/models.py:909: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means t

torch.Size([1, 32, 28, 28])
torch.Size([1, 24, 56, 56])
torch.Size([1, 16, 112, 112])
torch.Size([1, 1, 224, 224])


/Users/dhritiman.sagar/opt/anaconda3/lib/python3.7/site-packages/torch/onnx/symbolic_helper.py:253: UserWarning: You are trying to export the model with onnx:Upsample for ONNX opset version 9. This operator might cause results to not match the expected results by PyTorch.
ONNX's Upsample/Resize operator did not match Pytorch's Interpolation until opset 11. Attributes to determine how to transform the input were added in onnx:Resize in opset 11 to support Pytorch's behavior (like coordinate_transformation_mode and nearest_mode).
We recommend using opset 11 and above for models using this operator. 
  "" + str(_export_onnx_opset_version) + ". "


graph(%images : Float(1:150528, 3:50176, 224:224, 224:1),
      %conv0.0.weight : Float(32:27, 3:9, 3:3, 3:1),
      %conv0.1.weight : Float(32:1),
      %conv0.1.bias : Float(32:1),
      %conv0.1.running_mean : Float(32:1),
      %conv0.1.running_var : Float(32:1),
      %conv1.conv.0.0.weight : Float(32:9, 1:9, 3:3, 3:1),
      %conv1.conv.0.1.weight : Float(32:1),
      %conv1.conv.0.1.bias : Float(32:1),
      %conv1.conv.0.1.running_mean : Float(32:1),
      %conv1.conv.0.1.running_var : Float(32:1),
      %conv1.conv.1.weight : Float(16:32, 32:1, 1:1, 1:1),
      %conv1.conv.2.weight : Float(16:1),
      %conv1.conv.2.bias : Float(16:1),
      %conv1.conv.2.running_mean : Float(16:1),
      %conv1.conv.2.running_var : Float(16:1),
      %conv2.conv.0.0.weight : Float(96:16, 16:1, 1:1, 1:1),
      %conv2.conv.0.1.weight : Float(96:1),
      %conv2.conv.0.1.bias : Float(96:1),
      %conv2.conv.0.1.running_mean : Float(96:1),
      %conv2.conv.0.1.running_var : Float(96:1),
      

In [20]:
dummy_input = torch.randn(1, 3, 224, 224)

In [21]:
import PIL
from PIL import Image
import torchvision.transforms.functional as F
import onnxruntime as ort

In [22]:
img = Image.open('test_image.jpeg')
img = F.to_tensor(F.resize(img, (224, 224), interpolation=Image.BILINEAR)).unsqueeze(0).numpy()

In [23]:
session = ort.InferenceSession('depth_test.onnx')

In [24]:
outputs = session.run(None, {'images': img})

In [25]:
depth = outputs[0].reshape(224, 224)

In [ ]:
plt.imshow(depth, cmap='gray')

In [ ]:
plt.imshow(Image.open('test_image.jpeg'))

In [23]:
import cv2

True

In [24]:
depth_normalized = np.uint8((depth - np.min(depth))/(np.max(depth) - np.min(depth)) * 255)

In [25]:
cv2.imwrite('depth_test.png', depth_normalized)

True